In [ ]:
#imports
import whisper
from pytube import YouTube
from IPython.display import Markdown, display
import os
import warnings
from langchain_openai import ChatOpenAI

warnings.filterwarnings("ignore")

In [ ]:
def text_extraction(file, is_link=False, model_name="small", display_text=False):
    """
    Extracts text from an audio file or YouTube video link using the specified Whisper model.

    Arguments
    ---------
    file : str
        The path to the audio file or the YouTube video link.
    is_link : bool, optional
        Whether the input is a YouTube video link. The default is False.
    model_name : str, optional
        The name of the Whisper model to use for transcription. The default is "small".
    display_text : bool, optional
        Whether to display the extracted text. The default is False.
    
    Returns
    -------
    str
        The extracted text.
    """
    try:
        # Load the appropriate Whisper model
        model = whisper.load_model(model_name)
        
        if is_link:
            yt = YouTube(file)
            audio_stream = yt.streams.filter(only_audio=True).first()
            if not audio_stream:
                raise Exception("No audio stream found in the video.")
            
            audio_file = audio_stream.download(filename='audio.mp4')
            file = audio_file  # Update file variable to the downloaded audio file

        # Perform transcription
        result = model.transcribe(file)

        if display_text:
            display(Markdown(result["text"]))

        # Clean up the downloaded audio file if it's a link
        if is_link:
            os.remove(audio_file)

        return result["text"]

    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

# Example usage with a YouTube link
ted_talk_url = "https://www.youtube.com/watch?v=z7e7gtU3PHY"
text = text_extraction(ted_talk_url, is_link=True, model_name="small", display_text=True)

In [ ]:
	
# store text in a .txt file
with open("transcript.txt", "w") as file:
    file.write(text)

In [ ]:
def GPT_summarize(file_path, model='gpt-3.5-turbo', display_text=False):
    """
    Extracts text from an audio file or a YouTube video link using a specified Whisper model.

    Arguments
    ---------
    file : str
        The path to the audio file or the YouTube video link.
    is_link : bool, optional
        Indicates if 'file' is a YouTube video link. If True, the video's audio will be downloaded for processing.
        The default is False.
    model_name : str, optional
        Specifies the Whisper model to use for transcription. Valid options include "tiny", "base", "small", 
        "medium", and "large". The default is "small".
    display_text : bool, optional
        If True, displays the extracted text using Markdown format. The default is False.

    Returns
    -------
    str
        The extracted text as a string. Returns an empty string and prints an error message in case of failure.
    """

    # Read the text from the file
    with open(file_path, 'r') as file:
        text = file.read()

    prompt = f"Summarize the main topic of this text in one or two sentences:\n\n{text}"

    # Access ChatOpenAI to summarize the text
    llm = ChatOpenAI(temperature=0.1, model = model)
    summary = llm.invoke(prompt)

    # Display the summarized text if requested
    if display_text:
        display(Markdown(summary.content))

    return summary.content

# Example usage

# main topic
print("Summary:")
summary = GPT_summarize("transcript.txt", display_text=True)

In [ ]:
from langchain_openai import ChatOpenAI

def GPT_summarize(file_path, summary_style='main_topic', model='gpt-3.5-turbo', display_text=False):
    """
    Summarizes the contents of a text file using a specified OpenAI model.

    Arguments:
    ---------
    file_path : str
        Path to the text file to be summarized.
    summary_style : str, optional
        Type of summary needed: 'main_topic', 'abridged', or 'descriptive'.
        Default is 'main_topic'.
    model : str, optional
        Model to use for the AI summarization, options are 'gpt-3.5-turbo' or 'gpt-4-turbo'.
        Default is 'gpt-3.5-turbo'.
    display_text : bool, optional
        If True, displays the summarized text using Markdown format. Default is False.

    Returns:
    --------
    str
        The summarized text. If an invalid summary style is provided, a ValueError is raised.

    Notes:
    -----
    The function relies on the `langchain_openai.ChatOpenAI` class to interface with OpenAI's API.
    """

    # Read the text from the file
    with open(file_path, 'r') as file:
        text = file.read()

    # Generate the prompt based on the desired summary style
    if summary_style == 'main_topic':
        prompt = f"Summarize the main topic of this text in one or two sentences:\n\n{text}"
    elif summary_style == 'abridged':
        prompt = f"Provide an abridged summary of this text in 5-10 bullet points:\n\n{text}"
    elif summary_style == 'descriptive':
        prompt = f"Write a detailed summary of this text with a character limit of 3000 to 3500 characters:\n\n{text}"
    else:
        raise ValueError("Invalid summary style specified. Choose 'main_topic', 'abridged', or 'descriptive'.")

    # Access ChatOpenAI to summarize the text
    llm = ChatOpenAI(temperature=0.1, model = model)
    summary = llm.invoke(prompt)

    # Display the summarized text if requested
    if display_text:
        display(Markdown(summary.content))

    return summary.content

# Example usage

# main topic
print("Main Topic Summary:")
summary = GPT_summarize("transcript.txt", summary_style="main_topic", display_text=True)

# bullet points
print("\nAbridged Summary:")
summary = GPT_summarize("transcript.txt", summary_style="abridged", display_text=True)

# descriptive
print("\nDescriptive Summary:")
summary = GPT_summarize("transcript.txt", summary_style="descriptive", display_text=True)

In [ ]:
class AudioToSummary:

    def __init__(self, link, is_link, model_name_whisper='small', model_name_llm='gpt-3.5-turbo'):
        """
        Initialize the AudioToSummary instance with links and model specifications.

        Parameters
        ----------
        link : str
            The URL or path to the audio file to be processed. This could be a local path or a URL to a YouTube video.
        is_link : bool
            A flag to indicate whether the 'link' parameter is a URL to a YouTube video or a local file path.
        model_name_whisper : str, optional
            The name of the Whisper model to use for transcription. Defaults to 'small'.
        model_name_llm : str, optional
            The name of the language model to use for generating summaries. Defaults to 'gpt-3.5-turbo'.

        Attributes
        ----------
        text : str
            The text extracted from the audio file or YouTube video.
        _summary : str or None
            Cached value of the main topic summary. Calculated lazily.
        _description : str or None
            Cached value of the descriptive summary. Calculated lazily.
        _abridged_summary : str or None
            Cached value of the abridged summary. Calculated lazily.

        Notes
        -----
        The attributes `_summary`, `_description`, and `_abridged_summary` are lazy-loaded, meaning
        they are only computed when first accessed and if `text` is successfully extracted.
        """
        self.model_name_whisper = model_name_whisper
        self.link = link
        self.model_name_llm = model_name_llm
        self.is_link = is_link
        self.text = self.text_extraction(self.link, is_link)
        self._summary = None
        self._description = None
        self._abridged_summary = None

    @property
    def summary(self):
        """
        Retrieve or compute the main topic summary of the text.

        Returns
        -------
        str
            The main topic summary of the text. If `text` is not loaded, it returns None.
        """
        if self._summary is None and self.text:  # Generate summary only if text is loaded and summary not already generated
            self._summary = self.GPT_summarize(self.text)
        return self._summary

    @property
    def description(self):
        """
        Retrieve or compute the descriptive summary of the text.

        Returns
        -------
        str
            The descriptive summary of the text. If `text` is not loaded, it returns None.
        """
        if self._description is None and self.text:  # Generate descriptive summary only if text is loaded and not already generated
            self._description = self.GPT_summarize(self.text, summary_style='descriptive')
        return self._description

    @property
    def abridged_summary(self):
        """
        Retrieve or compute the abridged summary of the text.

        Returns
        -------
        str
            The abridged summary of the text. If `text` is not loaded, it returns None.
        """
        if self._abridged_summary is None and self.text:  # Generate abridged summary only if text is loaded and not already generated
            self._abridged_summary = self.GPT_summarize(self.text, summary_style='abridged_summary')
        return self._abridged_summary


    def text_extraction(self, file, is_link=False):
        """
        Extracts text from an audio file or YouTube video link using the specified Whisper model.

        Parameters:
        -----------
        file : str
            The path to the audio file or the YouTube video link.
        is_link : bool
            Specifies whether the input is a YouTube video link. Defaults to False.

        Returns:
        --------
        str
            The extracted text if successful, otherwise returns an empty string if an error occurs.

        Notes:
        -----
        Handles the downloading and deletion of the audio file if the source is a YouTube link.
        """

        try:
            model = whisper.load_model(self.model_name_whisper)
            if is_link:
                yt = YouTube(file)
                audio_stream = yt.streams.filter(only_audio=True).first()
                if not audio_stream:
                    raise Exception("No audio stream found in the video.")

                audio_file = audio_stream.download(filename='audio.mp4')
                file = audio_file  # Update file variable to the downloaded audio file

            result = model.transcribe(file)
            text = result["text"]

            if is_link:
                os.remove(audio_file)

            return text

        except Exception as e:
            print(f"An error occurred: {e}")
            return ""

    def GPT_summarize(self, text, summary_style='main_topic', model='gpt-3.5-turbo'):
        """
        Summarizes the text using a specified OpenAI model.

        Parameters:
        -----------
        text : str
            The text to be summarized.
        summary_style : str, optional
            The type of summary required: 'main_topic', 'abridged_summary', or 'descriptive'.
            Defaults to 'main_topic'.
        model : str, optional
            The model to use for the AI summarization, with options including 'gpt-3.5-turbo' or 'gpt-4-turbo'.
            Defaults to 'gpt-3.5-turbo'.

        Returns:
        --------
        str
            The summarized text. Raises a ValueError if an invalid summary style is provided.

        Notes:
        -----
        Leverages the OpenAI's GPT model for generating summaries based on the specified style.
        """

        # Generate the prompt based on the desired summary style
        if summary_style == 'main_topic':
            prompt = f"Summarize the main topic of this text in one or two sentences:\n\n{text}"
        elif summary_style == 'abridged_summary':
            prompt = f"Provide an abridged summary of this text in 5-10 bullet points:\n\n{text}"
        elif summary_style == 'descriptive':
            prompt = f"Write a detailed summary of this text with a character limit of 3000 to 3500 characters:\n\n{text}"
        else:
            raise ValueError("Invalid summary style specified. Choose 'main_topic', 'abridged', or 'descriptive'.")

        # Access ChatOpenAI to summarize the text
        llm = ChatOpenAI(temperature=0.1, model = model)
        summary = llm.invoke(prompt)

        return summary.content

In [ ]:
class AudioToSummary:

    def __init__(self, link, is_link, model_name_whisper='small', model_name_llm='gpt-3.5-turbo'):
        """
        Initialize the AudioToSummary instance with links and model specifications.

        Parameters
        ----------
        link : str
            The URL or path to the audio file to be processed. This could be a local path or a URL to a YouTube video.
        is_link : bool
            A flag to indicate whether the 'link' parameter is a URL to a YouTube video or a local file path.
        model_name_whisper : str, optional
            The name of the Whisper model to use for transcription. Defaults to 'small'.
        model_name_llm : str, optional
            The name of the language model to use for generating summaries. Defaults to 'gpt-3.5-turbo'.

        Attributes
        ----------
        text : str
            The text extracted from the audio file or YouTube video.
        _summary : str or None
            Cached value of the main topic summary. Calculated lazily.
        _description : str or None
            Cached value of the descriptive summary. Calculated lazily.
        _abridged_summary : str or None
            Cached value of the abridged summary. Calculated lazily.

        Notes
        -----
        The attributes `_summary`, `_description`, and `_abridged_summary` are lazy-loaded, meaning
        they are only computed when first accessed and if `text` is successfully extracted.
        """
        self.model_name_whisper = model_name_whisper
        self.link = link
        self.model_name_llm = model_name_llm
        self.is_link = is_link
        self.text = self.text_extraction(self.link, is_link)
        self._summary = None
        self._description = None
        self._abridged_summary = None

    @property
    def summary(self):
        """
        Retrieve or compute the main topic summary of the text.

        Returns
        -------
        str
            The main topic summary of the text. If `text` is not loaded, it returns None.
        """
        if self._summary is None and self.text:  # Generate summary only if text is loaded and summary not already generated
            self._summary = self.GPT_summarize(self.text)
        return self._summary

    @property
    def description(self):
        """
        Retrieve or compute the descriptive summary of the text.

        Returns
        -------
        str
            The descriptive summary of the text. If `text` is not loaded, it returns None.
        """
        if self._description is None and self.text:  # Generate descriptive summary only if text is loaded and not already generated
            self._description = self.GPT_summarize(self.text, summary_style='descriptive')
        return self._description

    @property
    def abridged_summary(self):
        """
        Retrieve or compute the abridged summary of the text.

        Returns
        -------
        str
            The abridged summary of the text. If `text` is not loaded, it returns None.
        """
        if self._abridged_summary is None and self.text:  # Generate abridged summary only if text is loaded and not already generated
            self._abridged_summary = self.GPT_summarize(self.text, summary_style='abridged_summary')
        return self._abridged_summary


    def text_extraction(self, file, is_link=False):
        """
        Extracts text from an audio file or YouTube video link using the specified Whisper model.

        Parameters:
        -----------
        file : str
            The path to the audio file or the YouTube video link.
        is_link : bool
            Specifies whether the input is a YouTube video link. Defaults to False.

        Returns:
        --------
        str
            The extracted text if successful, otherwise returns an empty string if an error occurs.

        Notes:
        -----
        Handles the downloading and deletion of the audio file if the source is a YouTube link.
        """

        try:
            model = whisper.load_model(self.model_name_whisper)
            if is_link:
                yt = YouTube(file)
                audio_stream = yt.streams.filter(only_audio=True).first()
                if not audio_stream:
                    raise Exception("No audio stream found in the video.")

                audio_file = audio_stream.download(filename='audio.mp4')
                file = audio_file  # Update file variable to the downloaded audio file

            result = model.transcribe(file)
            text = result["text"]

            if is_link:
                os.remove(audio_file)

            return text

        except Exception as e:
            print(f"An error occurred: {e}")
            return ""

    def GPT_summarize(self, text, summary_style='main_topic', model='gpt-3.5-turbo'):
        """
        Summarizes the text using a specified OpenAI model.

        Parameters:
        -----------
        text : str
            The text to be summarized.
        summary_style : str, optional
            The type of summary required: 'main_topic', 'abridged_summary', or 'descriptive'.
            Defaults to 'main_topic'.
        model : str, optional
            The model to use for the AI summarization, with options including 'gpt-3.5-turbo' or 'gpt-4-turbo'.
            Defaults to 'gpt-3.5-turbo'.

        Returns:
        --------
        str
            The summarized text. Raises a ValueError if an invalid summary style is provided.

        Notes:
        -----
        Leverages the OpenAI's GPT model for generating summaries based on the specified style.
        """

        # Generate the prompt based on the desired summary style
        if summary_style == 'main_topic':
            prompt = f"Summarize the main topic of this text in one or two sentences:\n\n{text}"
        elif summary_style == 'abridged_summary':
            prompt = f"Provide an abridged summary of this text in 5-10 bullet points:\n\n{text}"
        elif summary_style == 'descriptive':
            prompt = f"Write a detailed summary of this text with a character limit of 3000 to 3500 characters:\n\n{text}"
        else:
            raise ValueError("Invalid summary style specified. Choose 'main_topic', 'abridged', or 'descriptive'.")

        # Access ChatOpenAI to summarize the text
        llm = ChatOpenAI(temperature=0.1, model = model)
        summary = llm.invoke(prompt)

        return summary.content
# Example 1
processor = AudioToSummary(link ="https://www.youtube.com/watch?v=z7e7gtU3PHY",is_link=True)
print('summary: ', processor.summary)
print()
print('description: ', processor.description)
print()
print('Abridget summary: ', processor.abridged_summary)

In [ ]:
#Example 2
Veritasium = AudioToSummary(link ="https://www.youtube.com/watch?v=A5w-dEgIU1M",is_link=True)
print('summary: ', Veritasium.summary)
print()
print('description: ', Veritasium.description)
print()
print('Abridget summary: ', Veritasium.abridged_summary)